# Import

In [8]:
from functools import reduce
from pathlib import Path
import time
import datetime

import pandas as pd
import requests
from wmfdata import hive, spark
from wmfdata.utils import print_err, pd_display_all

import warnings
warnings.filterwarnings('ignore')


# Parameters

In [9]:
# TSV file where metrics are or will be saved
FILENAME = "metrics/readers_metrics.tsv"

# Metric month. The mediawiki_history snapshot must be from the metrics month or later.
last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

METRICS_MONTH_TEXT = last_month.strftime("%Y-%m")
MEDIAWIKI_HISTORY_SNAPSHOT = last_month.strftime("%Y-%m")



In [10]:
datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

datetime.date(2023, 8, 31)

# Preparation

In [11]:
# Convert our metrics month to all the formats we need and provide them in a dict
# so we can easily use them to format strings
metrics_month = METRICS_MONTH_TEXT
date_params = {
    "mediawiki_history_snapshot": MEDIAWIKI_HISTORY_SNAPSHOT,
    "metrics_month": str(metrics_month),
    #"metrics_prev_month": str(metrics_month - 1),
    #"metrics_month_start": str(metrics_month.start_time), 
    "metrics_month_first_day": str((datetime.date.today()- datetime.timedelta(days=31)).replace(day=1)),
    #"metrics_month_end": str(last_month),
    "metrics_month_last_day": str(last_month),
    "metrics_year": last_month.year,
    "metrics_cur_month" : last_month.month
}

# Load any previous results
try:
    old_metrics = (
        pd.read_csv(FILENAME, sep="\t", parse_dates = ["month"])
        .set_index("month")
    )
except FileNotFoundError:
    old_metrics = None
    
def prepare_query(filename):
    return (
        Path(filename)
        .read_text()
        .format(**date_params)
    )

In [12]:
date_params

{'mediawiki_history_snapshot': '2023-08',
 'metrics_month': '2023-08',
 'metrics_month_first_day': '2023-08-01',
 'metrics_month_last_day': '2023-08-31',
 'metrics_year': 2023,
 'metrics_cur_month': 8}

# MariaDB and Hive query metrics


In [13]:
# Code to Suppress PySpark Warning messages
from warnings import filterwarnings
filterwarnings("ignore")

In [14]:
queries = {
    "pageviews": {
        "file": "queries/pageviews.hql",
        "engine": "hive"
    },
    
    "automated_pageviews": {
        "file": "queries/automated_pageviews.hql",
        "engine": "hive"
    },
    "page_previews": {
        "file": "queries/page_previews.hql",
        "engine": "hive"
    },
    "unique_devices": {
        "file": "queries/unique_devices.hql",
        "engine": "hive"
    }


# Removing since global south metrics are calculated in notebook 02b-diversity-calculation.ipynb
   # ,
   # "global_south_pageviews": {
   #     "file": "queries/global_south_pageviews.hql",
   #     "engine": "hive"
   # },
    
   # "global_south_previews": {
   #     "file": "queries/global_south_previews.hql",
   #     "engine": "hive"
   # }
    
# Removing since mobile-heavy metrics are no longer tracked/reported on in the key product metrics presentations
    #"mobile-heavy_pageviews": {
    #    "file": "queries/mobile_heavy_pageviews.hql",
    #    "engine": "hive" }, 
    #"mobile-heavy_previews": {
    #   "file": "queries/mobile_heavy_previews.hql",
    #   "engine": "hive" }
    
       
}


for key, val in queries.items():
    query = prepare_query(val["file"])
    engine = val["engine"]
    print_err("Running {} on {}...".format(key, engine))
    
    if engine == "mariadb":
        result = mariadb.run(query)
    elif engine == "hive":
        result = spark.run(query)
    else:
        raise ValueError("Unknown engine specified.") 
    
    result = result.assign(month=lambda df: pd.to_datetime(df["month"]))
    val["result"] = result

Running pageviews on hive...



SPARK_HOME: /usr/lib/spark3
Using Hadoop client lib jars at 3.2.0, provided by Spark.
PYSPARK_PYTHON=/opt/conda-analytics/bin/python3


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/21 23:03:21 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/09/21 23:03:34 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!
23/09/21 23:03:39 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
Running automated_pageviews on hive...                                          


Running page_previews on hive...                                                


Running unique_devices on hive...                                               


# Combining and saving metrics

In [15]:
# Assemble list of result dataframes
results = [val["result"] for _, val in queries.items()]

# Merge them all, assuming that the month is the only common column
new_metrics = reduce(lambda l, r: pd.merge(l, r, how="outer"), results)

# Set the month as an index so combine_first works properly
new_metrics = new_metrics.set_index("month").sort_index()

# Add Metrics for Interactions
new_metrics['interactions'] = new_metrics.apply(lambda x: x['previews_seen'] + x['total_pageview'], axis=1)


In [16]:
if old_metrics is None:
    metrics = new_metrics
else:
    metrics = new_metrics.combine_first(old_metrics)
    
pd_display_all(metrics.tail(10))

,automated_pageviews,desktop,interactions,mobileweb,previews_seen,total_pageview,unique_devices
month,,,,,,,
2022-11-01,3.125762e+09,6639885018,2.103694e+10,12391464131,1.725951e+09,19310984429,1.602483e+09
2022-12-01,1.514616e+09,6245363227,1.853431e+10,10336993739,1.645032e+09,16889273891,1.580337e+09
2023-01-01,2.058892e+09,6655879078,2.052910e+10,11696154738,1.853753e+09,18675346647,1.623196e+09
2023-02-01,2.484894e+09,5809895389,1.896141e+10,11232689898,1.643540e+09,17317864773,1.556624e+09
2023-03-01,2.851128e+09,6554145894,2.110827e+10,12446354588,1.808851e+09,19299418022,1.666366e+09
2023-04-01,1.835848e+09,5903055420,1.857836e+10,10744497885,1.635800e+09,16942562523,1.653133e+09
2023-05-01,2.788857e+09,6136981565,2.086826e+10,12693678053,1.738922e+09,19129336129,1.596438e+09
2023-06-01,2.535726e+09,5642808989,1.956977e+10,12093286214,1.548887e+09,18020883276,1.522910e+09
2023-07-01,2.765634e+09,5557997700,2.010882e+10,12691626287,1.547944e+09,18560871345,1.515395e+09


In [17]:
metrics.to_csv(FILENAME, sep="\t")